In [1]:
import shutil, os, csv, itertools, glob

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from dataloader import train_loader, val_loader

Loading data...
Done
Loading data...
Done


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%load_ext autoreload
%autoreload 1
%aimport configs
cfgs = configs
cuda = cfgs.USE_CUDA
%aimport model
CAN = model.CAN

In [ ]:
def get_acc(output, target):
    # takes in two tensors to compute accuracy
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct = pred.eq(target.data.view_as(pred)).cpu().sum()
#     print("Output: ")
#     print(output.data.squeeze().cpu().numpy())
#     print("Pred: ")
#     print(pred.squeeze().cpu().numpy())
#     print("Target: ")
#     print(target.data.cpu().numpy())
    return correct, target.size(0)

def run_trainer(model_path, model, train_loader, test_loader, get_acc, resume, num_epoch):

    if not os.path.exists(model_path):
        os.makedirs(model_path)
    def save_checkpoint(state, is_best, filename=model_path+'checkpoint.pth.tar'):
        torch.save(state, filename)
        if is_best:
            shutil.copyfile(filename, model_path+'model_best.pth.tar')
    def get_last_checkpoint(model_path):
        fs = sorted([f for f in os.listdir(model_path) if 'Epoch' in f], key=lambda k: int(k.split()[1]))
        return model_path+fs[-1] if len(fs) > 0 else None
    
    start_epoch = 0
    best_res = 0
    resume_state = get_last_checkpoint(model_path) if resume else None
    if resume_state and os.path.isfile(resume_state):
        print("=> loading checkpoint '{}'".format(resume_state))
        checkpoint = torch.load(resume_state)
        start_epoch = checkpoint['epoch']+1
        best_res = checkpoint['val_acc']
        model.load_state_dict(checkpoint['state_dict'])
        if cuda:
            model.cuda()
        optimizer = optim.Adam(model.parameters(), **cfgs.OPT_PARAM)
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resume_state, checkpoint['epoch']))
    else:
        if cuda:
            model.cuda()
        optimizer = optim.Adam(model.parameters(), **cfgs.OPT_PARAM)

    criterion = nn.CrossEntropyLoss()
    # scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5) # optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5)

    def train(epoch):
        model.train()
        total, total_correct = 0., 0.
        for batch_idx, (img_feats, question, answer, lengths) in enumerate(train_loader):
            img_feats, question, answer = Variable(img_feats.float()), Variable(question.long()), Variable(answer.long())
            if cuda:
                img_feats, question, answer = img_feats.cuda(), question.cuda(), answer.cuda()
            optimizer.zero_grad()
            output = model(img_feats, question, lengths)
            loss = criterion(output, answer)
            loss.backward()
            optimizer.step()

            correct, num_instance = get_acc(output, answer)
            total_correct += correct
            total += num_instance
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Acc: {:.2f}%/{:.2f}%'.format(
                    epoch, batch_idx * cfgs.BATCH_SIZE, len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0],
                    100. * correct / num_instance, 100. * total_correct / total ))
        
        return 100. * total_correct / total

    def test():
        model.eval()
        test_loss = 0.
        total, total_correct = 0., 0.
        for img_feats, question, answer, lengths in test_loader:
            img_feats, question, answer = Variable(img_feats.float()), Variable(question.long()), Variable(answer.long())
            if cuda:
                img_feats, question, answer = img_feats.cuda(), question.cuda(), answer.cuda()
            optimizer.zero_grad()
            output = model(img_feats, question, lengths)
            test_loss += criterion(output, answer).data[0] # sum up batch loss
            
            correct, num_instance = get_acc(output, answer)
            total_correct += correct
            total += num_instance

        test_acc = 100. * total_correct / total
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, total_correct, total,
            test_acc))

        return test_acc


    for epoch in range(start_epoch, num_epoch):
        is_best = False

        train_acc = train(epoch)
        val_acc = test()
        
        # scheduler.step(val_loss)

        if val_acc > best_res:
            best_res = val_acc
            is_best = True

        save_checkpoint({
                'epoch': epoch,
                'state_dict': model.cpu().state_dict(),
                'train_acc':train_acc,
                'val_acc': val_acc,
                'optimizer' : optimizer.state_dict(),
                'net_cfg':cfgs.NET_PARAM,
            }, is_best,
            model_path+"Epoch %d Acc %.4f.pt"%(epoch, val_acc))

        if cuda:
            model.cuda()

In [ ]:
print(cfgs.NET_PARAM)
model = CAN(**cfgs.NET_PARAM)
run_trainer(
    model_path = './ckpt_p12_new/', 
    model = model, 
    train_loader = train_loader, 
    test_loader = val_loader, 
    get_acc = get_acc, 
    resume = False, 
    num_epoch = 100
)

{'vocab_size': 81, 'write_mem_gate': False, 'input_hidden_channels': None, 'embedding_dim': 300, 'out_hidden_size': 512, 'd': 512, 'p': 12, 'num_answers': 28, 'write_self_attn': False, 'input_in_channels': 1024}
Train Epoch: 0 [0/699989 (0%)]	Loss: 3.396354 Acc: 4.69%/4.69%
Train Epoch: 0 [640/699989 (0%)]	Loss: 2.991068 Acc: 18.75%/10.51%
Train Epoch: 0 [1280/699989 (0%)]	Loss: 2.718542 Acc: 17.19%/14.88%
Train Epoch: 0 [1920/699989 (0%)]	Loss: 2.769975 Acc: 20.31%/17.44%
Train Epoch: 0 [2560/699989 (0%)]	Loss: 2.610999 Acc: 18.75%/18.10%
Train Epoch: 0 [3200/699989 (0%)]	Loss: 2.269485 Acc: 31.25%/19.79%
Train Epoch: 0 [3840/699989 (1%)]	Loss: 2.179698 Acc: 21.88%/20.11%
Train Epoch: 0 [4480/699989 (1%)]	Loss: 1.972322 Acc: 26.56%/21.08%
Train Epoch: 0 [5120/699989 (1%)]	Loss: 1.739567 Acc: 37.50%/22.74%
Train Epoch: 0 [5760/699989 (1%)]	Loss: 1.830918 Acc: 28.12%/23.51%
Train Epoch: 0 [6400/699989 (1%)]	Loss: 1.616213 Acc: 42.19%/24.32%
Train Epoch: 0 [7040/699989 (1%)]	Loss: 1.5557

Train Epoch: 0 [85120/699989 (12%)]	Loss: 0.928524 Acc: 51.56%/40.45%
Train Epoch: 0 [85760/699989 (12%)]	Loss: 0.911389 Acc: 46.88%/40.49%
Train Epoch: 0 [86400/699989 (12%)]	Loss: 1.053518 Acc: 31.25%/40.51%
Train Epoch: 0 [87040/699989 (12%)]	Loss: 1.021132 Acc: 46.88%/40.52%
Train Epoch: 0 [87680/699989 (13%)]	Loss: 1.056814 Acc: 46.88%/40.54%
Train Epoch: 0 [88320/699989 (13%)]	Loss: 1.116144 Acc: 29.69%/40.55%
Train Epoch: 0 [88960/699989 (13%)]	Loss: 1.052788 Acc: 42.19%/40.55%
Train Epoch: 0 [89600/699989 (13%)]	Loss: 0.988529 Acc: 45.31%/40.60%
Train Epoch: 0 [90240/699989 (13%)]	Loss: 0.967519 Acc: 37.50%/40.59%
Train Epoch: 0 [90880/699989 (13%)]	Loss: 0.987197 Acc: 39.06%/40.59%
Train Epoch: 0 [91520/699989 (13%)]	Loss: 0.946942 Acc: 48.44%/40.61%
Train Epoch: 0 [92160/699989 (13%)]	Loss: 1.043105 Acc: 45.31%/40.61%
Train Epoch: 0 [92800/699989 (13%)]	Loss: 1.162843 Acc: 42.19%/40.63%
Train Epoch: 0 [93440/699989 (13%)]	Loss: 1.028226 Acc: 50.00%/40.66%
Train Epoch: 0 [9408

Train Epoch: 0 [159360/699989 (23%)]	Loss: 1.107024 Acc: 37.50%/41.55%
Train Epoch: 0 [160000/699989 (23%)]	Loss: 1.146758 Acc: 40.62%/41.56%
Train Epoch: 0 [160640/699989 (23%)]	Loss: 0.974724 Acc: 42.19%/41.56%
Train Epoch: 0 [161280/699989 (23%)]	Loss: 1.160323 Acc: 32.81%/41.57%
Train Epoch: 0 [161920/699989 (23%)]	Loss: 0.936450 Acc: 35.94%/41.58%
Train Epoch: 0 [162560/699989 (23%)]	Loss: 0.897063 Acc: 39.06%/41.58%
Train Epoch: 0 [163200/699989 (23%)]	Loss: 1.065238 Acc: 37.50%/41.58%
Train Epoch: 0 [163840/699989 (23%)]	Loss: 1.060459 Acc: 31.25%/41.59%
Train Epoch: 0 [164480/699989 (23%)]	Loss: 1.016955 Acc: 50.00%/41.60%
Train Epoch: 0 [165120/699989 (24%)]	Loss: 1.005218 Acc: 46.88%/41.60%
Train Epoch: 0 [165760/699989 (24%)]	Loss: 0.978644 Acc: 34.38%/41.59%
Train Epoch: 0 [166400/699989 (24%)]	Loss: 0.979318 Acc: 37.50%/41.60%
Train Epoch: 0 [167040/699989 (24%)]	Loss: 1.072242 Acc: 35.94%/41.60%
Train Epoch: 0 [167680/699989 (24%)]	Loss: 1.028942 Acc: 39.06%/41.59%
Train 

Train Epoch: 0 [233600/699989 (33%)]	Loss: 0.972745 Acc: 46.88%/42.21%
Train Epoch: 0 [234240/699989 (33%)]	Loss: 1.015514 Acc: 35.94%/42.21%
Train Epoch: 0 [234880/699989 (34%)]	Loss: 0.974745 Acc: 48.44%/42.21%
Train Epoch: 0 [235520/699989 (34%)]	Loss: 1.062412 Acc: 42.19%/42.21%
Train Epoch: 0 [236160/699989 (34%)]	Loss: 0.977521 Acc: 37.50%/42.22%
Train Epoch: 0 [236800/699989 (34%)]	Loss: 1.029232 Acc: 42.19%/42.23%
Train Epoch: 0 [237440/699989 (34%)]	Loss: 0.880309 Acc: 46.88%/42.24%
Train Epoch: 0 [238080/699989 (34%)]	Loss: 0.951351 Acc: 48.44%/42.24%
Train Epoch: 0 [238720/699989 (34%)]	Loss: 1.034484 Acc: 48.44%/42.25%
Train Epoch: 0 [239360/699989 (34%)]	Loss: 1.011502 Acc: 40.62%/42.26%
Train Epoch: 0 [240000/699989 (34%)]	Loss: 1.028736 Acc: 37.50%/42.26%
Train Epoch: 0 [240640/699989 (34%)]	Loss: 1.113776 Acc: 40.62%/42.27%
Train Epoch: 0 [241280/699989 (34%)]	Loss: 0.932411 Acc: 42.19%/42.27%
Train Epoch: 0 [241920/699989 (35%)]	Loss: 1.059688 Acc: 35.94%/42.28%
Train 

Train Epoch: 0 [307840/699989 (44%)]	Loss: 0.952656 Acc: 51.56%/42.74%
Train Epoch: 0 [308480/699989 (44%)]	Loss: 0.886282 Acc: 51.56%/42.74%
Train Epoch: 0 [309120/699989 (44%)]	Loss: 0.950664 Acc: 50.00%/42.75%
Train Epoch: 0 [309760/699989 (44%)]	Loss: 1.143092 Acc: 39.06%/42.75%
Train Epoch: 0 [310400/699989 (44%)]	Loss: 1.073298 Acc: 43.75%/42.76%
Train Epoch: 0 [311040/699989 (44%)]	Loss: 0.917092 Acc: 35.94%/42.77%
Train Epoch: 0 [311680/699989 (45%)]	Loss: 0.921485 Acc: 35.94%/42.77%
Train Epoch: 0 [312320/699989 (45%)]	Loss: 0.921221 Acc: 42.19%/42.77%
Train Epoch: 0 [312960/699989 (45%)]	Loss: 1.020521 Acc: 34.38%/42.78%
Train Epoch: 0 [313600/699989 (45%)]	Loss: 1.085998 Acc: 48.44%/42.79%
Train Epoch: 0 [314240/699989 (45%)]	Loss: 1.034157 Acc: 48.44%/42.79%
Train Epoch: 0 [314880/699989 (45%)]	Loss: 0.909944 Acc: 51.56%/42.79%
Train Epoch: 0 [315520/699989 (45%)]	Loss: 0.967511 Acc: 39.06%/42.80%
Train Epoch: 0 [316160/699989 (45%)]	Loss: 0.805555 Acc: 53.12%/42.81%
Train 

...

Train Epoch: 11 [588800/699989 (84%)]	Loss: 0.085913 Acc: 95.31%/95.86%
Train Epoch: 11 [589440/699989 (84%)]	Loss: 0.104532 Acc: 95.31%/95.86%
Train Epoch: 11 [590080/699989 (84%)]	Loss: 0.064992 Acc: 96.88%/95.86%
Train Epoch: 11 [590720/699989 (84%)]	Loss: 0.049295 Acc: 98.44%/95.86%
Train Epoch: 11 [591360/699989 (84%)]	Loss: 0.062379 Acc: 96.88%/95.86%
Train Epoch: 11 [592000/699989 (85%)]	Loss: 0.110773 Acc: 92.19%/95.86%
Train Epoch: 11 [592640/699989 (85%)]	Loss: 0.061840 Acc: 98.44%/95.86%
Train Epoch: 11 [593280/699989 (85%)]	Loss: 0.074917 Acc: 96.88%/95.86%
Train Epoch: 11 [593920/699989 (85%)]	Loss: 0.091974 Acc: 95.31%/95.86%
Train Epoch: 11 [594560/699989 (85%)]	Loss: 0.161119 Acc: 90.62%/95.86%
Train Epoch: 11 [595200/699989 (85%)]	Loss: 0.229294 Acc: 93.75%/95.86%
Train Epoch: 11 [595840/699989 (85%)]	Loss: 0.161570 Acc: 92.19%/95.86%
Train Epoch: 11 [596480/699989 (85%)]	Loss: 0.154991 Acc: 92.19%/95.86%
Train Epoch: 11 [597120/699989 (85%)]	Loss: 0.090078 Acc: 96.88%

Train Epoch: 11 [661760/699989 (95%)]	Loss: 0.075368 Acc: 98.44%/95.85%
Train Epoch: 11 [662400/699989 (95%)]	Loss: 0.139925 Acc: 92.19%/95.85%
Train Epoch: 11 [663040/699989 (95%)]	Loss: 0.076840 Acc: 98.44%/95.85%
Train Epoch: 11 [663680/699989 (95%)]	Loss: 0.123211 Acc: 95.31%/95.85%
Train Epoch: 11 [664320/699989 (95%)]	Loss: 0.030897 Acc: 100.00%/95.85%
Train Epoch: 11 [664960/699989 (95%)]	Loss: 0.235764 Acc: 89.06%/95.85%
Train Epoch: 11 [665600/699989 (95%)]	Loss: 0.060698 Acc: 98.44%/95.85%
Train Epoch: 11 [666240/699989 (95%)]	Loss: 0.078378 Acc: 96.88%/95.85%
Train Epoch: 11 [666880/699989 (95%)]	Loss: 0.024875 Acc: 98.44%/95.85%
Train Epoch: 11 [667520/699989 (95%)]	Loss: 0.175808 Acc: 89.06%/95.85%
Train Epoch: 11 [668160/699989 (95%)]	Loss: 0.192995 Acc: 92.19%/95.85%
Train Epoch: 11 [668800/699989 (96%)]	Loss: 0.114105 Acc: 96.88%/95.85%
Train Epoch: 11 [669440/699989 (96%)]	Loss: 0.102028 Acc: 95.31%/95.85%
Train Epoch: 11 [670080/699989 (96%)]	Loss: 0.078314 Acc: 98.44

Train Epoch: 12 [35200/699989 (5%)]	Loss: 0.091083 Acc: 96.88%/96.39%
Train Epoch: 12 [35840/699989 (5%)]	Loss: 0.018646 Acc: 98.44%/96.40%
Train Epoch: 12 [36480/699989 (5%)]	Loss: 0.113855 Acc: 96.88%/96.42%
Train Epoch: 12 [37120/699989 (5%)]	Loss: 0.053427 Acc: 98.44%/96.43%
Train Epoch: 12 [37760/699989 (5%)]	Loss: 0.055191 Acc: 96.88%/96.42%
Train Epoch: 12 [38400/699989 (5%)]	Loss: 0.083722 Acc: 96.88%/96.43%
Train Epoch: 12 [39040/699989 (6%)]	Loss: 0.147153 Acc: 90.62%/96.40%
Train Epoch: 12 [39680/699989 (6%)]	Loss: 0.053664 Acc: 100.00%/96.40%
Train Epoch: 12 [40320/699989 (6%)]	Loss: 0.077552 Acc: 98.44%/96.41%
Train Epoch: 12 [40960/699989 (6%)]	Loss: 0.082088 Acc: 95.31%/96.40%
Train Epoch: 12 [41600/699989 (6%)]	Loss: 0.035682 Acc: 98.44%/96.42%
Train Epoch: 12 [42240/699989 (6%)]	Loss: 0.079657 Acc: 96.88%/96.44%
Train Epoch: 12 [42880/699989 (6%)]	Loss: 0.081247 Acc: 95.31%/96.44%
Train Epoch: 12 [43520/699989 (6%)]	Loss: 0.075986 Acc: 98.44%/96.44%
Train Epoch: 12 [44

Train Epoch: 12 [109440/699989 (16%)]	Loss: 0.064330 Acc: 98.44%/96.33%
Train Epoch: 12 [110080/699989 (16%)]	Loss: 0.177510 Acc: 93.75%/96.34%
Train Epoch: 12 [110720/699989 (16%)]	Loss: 0.030741 Acc: 98.44%/96.34%
Train Epoch: 12 [111360/699989 (16%)]	Loss: 0.061109 Acc: 98.44%/96.35%
Train Epoch: 12 [112000/699989 (16%)]	Loss: 0.175088 Acc: 90.62%/96.35%
Train Epoch: 12 [112640/699989 (16%)]	Loss: 0.026805 Acc: 100.00%/96.35%
Train Epoch: 12 [113280/699989 (16%)]	Loss: 0.077420 Acc: 96.88%/96.34%
Train Epoch: 12 [113920/699989 (16%)]	Loss: 0.055571 Acc: 96.88%/96.35%
Train Epoch: 12 [114560/699989 (16%)]	Loss: 0.050399 Acc: 98.44%/96.35%
Train Epoch: 12 [115200/699989 (16%)]	Loss: 0.135010 Acc: 95.31%/96.34%
Train Epoch: 12 [115840/699989 (17%)]	Loss: 0.054985 Acc: 96.88%/96.34%
Train Epoch: 12 [116480/699989 (17%)]	Loss: 0.112876 Acc: 96.88%/96.35%
Train Epoch: 12 [117120/699989 (17%)]	Loss: 0.063619 Acc: 96.88%/96.35%
Train Epoch: 12 [117760/699989 (17%)]	Loss: 0.186053 Acc: 95.31

Train Epoch: 12 [182400/699989 (26%)]	Loss: 0.131741 Acc: 92.19%/96.24%
Train Epoch: 12 [183040/699989 (26%)]	Loss: 0.097268 Acc: 96.88%/96.24%
Train Epoch: 12 [183680/699989 (26%)]	Loss: 0.083558 Acc: 93.75%/96.24%
Train Epoch: 12 [184320/699989 (26%)]	Loss: 0.238826 Acc: 90.62%/96.24%
Train Epoch: 12 [184960/699989 (26%)]	Loss: 0.053037 Acc: 98.44%/96.24%
Train Epoch: 12 [185600/699989 (27%)]	Loss: 0.279723 Acc: 90.62%/96.24%
Train Epoch: 12 [186240/699989 (27%)]	Loss: 0.066492 Acc: 98.44%/96.24%
Train Epoch: 12 [186880/699989 (27%)]	Loss: 0.053647 Acc: 96.88%/96.24%
Train Epoch: 12 [187520/699989 (27%)]	Loss: 0.022426 Acc: 98.44%/96.24%
Train Epoch: 12 [188160/699989 (27%)]	Loss: 0.024955 Acc: 100.00%/96.25%
Train Epoch: 12 [188800/699989 (27%)]	Loss: 0.077596 Acc: 95.31%/96.25%
Train Epoch: 12 [189440/699989 (27%)]	Loss: 0.043792 Acc: 98.44%/96.25%
Train Epoch: 12 [190080/699989 (27%)]	Loss: 0.141556 Acc: 95.31%/96.25%
Train Epoch: 12 [190720/699989 (27%)]	Loss: 0.031228 Acc: 98.44